In [11]:
import time                      # import libraries
start_time = time.time()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import imblearn
from imblearn.over_sampling import SMOTE
import collections
from tqdm import tqdm_notebook as tqdm

from sklearn.preprocessing import StandardScaler      # import sklearn libraries
from sklearn import datasets, model_selection
from sklearn.metrics import plot_confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix,plot_roc_curve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.naive_bayes import GaussianNB

from xgboost import XGBClassifier                     # import non-sklearn algorithm libraries
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


In [12]:
df = pd.read_csv('./DATA/creditcard_train1.csv')     # import train csv files to dataframes
df2 = pd.read_csv('./DATA/creditcard_train2.csv')
df = pd.concat([df,df2[1:]])                        # merge the 2 dataframes
del df2                                             # delete the one of the dataframes

--- 1.0 seconds ---


In [13]:
df.isnull().sum().sum()   # Check if there is any Nan in the dataframe

0

In [18]:
features_list = df.columns.values.tolist()  # create a list for the name of all the features
for list_idx in features_list:
    print(list_idx, end=' ')               # printing the list of features and label
features_list.pop()                        # deleting the list entity of the list as it is the label(not feature)
print("\n")
for list_idx in features_list:
    print(list_idx, end=' ')               # printing the list of only features

Time V1 V2 V3 V4 V5 V6 V7 V8 V9 V10 V11 V12 V13 V14 V15 V16 V17 V18 V19 V20 V21 V22 V23 V24 V25 V26 V27 V28 Amount Class 

Time V1 V2 V3 V4 V5 V6 V7 V8 V9 V10 V11 V12 V13 V14 V15 V16 V17 V18 V19 V20 V21 V22 V23 V24 V25 V26 V27 V28 Amount 

In [19]:
df.Class.unique()  # ensuring the Class column is binary as expected

array([0, 1], dtype=int64)

In [21]:
df.Class=df.Class.astype("bool")  # converting the Class label to boolean type from int64
collections.Counter(df.Class)       # checking how inbalance is our data

Counter({False: 198998, True: 364})

In [22]:
df.duplicated().sum()    # Check for duplication

585

In [24]:
df.drop_duplicates(inplace = True)  # keeping only one of the duplication rows if there is any
collections.Counter(df.Class)        # check again how our inbalance data looks like

Counter({False: 198422, True: 355})

In [25]:
fraud_count = df.Class[df["Class"]==True].count() # splittig the dataframe to fraud and clean transactions dataframes
clean_count = len(df) - fraud_count
print('Number of fraud Transactions is ', fraud_count ,' which is ', round(100*fraud_count/len(df),3) , "% of the dataset.\n")
print('Number of clean Transactions is ', clean_count ,' which is ', round(100*clean_count/len(df),3) , "% of the dataset.")

Number of fraud Transactions is  355  which is  0.179 % of the dataset.

Number of clean Transactions is  198422  which is  99.821 % of the dataset.


In [27]:
df_label=df["Class"]                    # creating a label only dataframe
df_features = df.drop("Class", axis=1)   # creating a feature only dataframe

In [28]:
# Separating the train and validation sets for both labels and feautures
X_train, X_val, Y_train, Y_val = train_test_split(df_features, df_label, test_size=0.33, random_state=42)

In [30]:
df.describe() # check the description of all the numeric features(which are all the features) 

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
count,198777.000000,198777.000000,198777.000000,198777.000000,198777.000000,198777.000000,198777.000000,198777.000000,198777.000000,198777.000000,...,198777.000000,198777.000000,198777.000000,198777.000000,198777.000000,198777.000000,198777.000000,198777.000000,198777.000000,198777.000000
mean,94804.312295,0.009128,-0.002808,-0.001230,-0.002365,-0.002014,0.002832,0.001295,-0.001479,0.001572,...,-0.000200,-0.000324,-0.001453,-0.000254,0.000687,-0.000352,0.000263,0.001165,0.000728,88.854125
std,47505.945585,1.945998,1.634265,1.513868,1.415270,1.386037,1.335744,1.243473,1.177720,1.099267,...,0.769537,0.725705,0.724582,0.631072,0.605287,0.521072,0.482129,0.399762,0.328130,253.999620
min,0.000000,-56.407510,-72.715728,-48.325589,-5.683171,-113.743307,-26.160506,-43.557242,-50.688419,-13.434066,...,-54.497720,-22.889347,-9.499423,-44.807735,-2.824849,-10.295397,-2.604551,-22.565679,-15.430084,0.000000
25%,54213.000000,-0.914367,-0.600632,-0.891180,-0.848943,-0.691728,-0.768274,-0.554928,-0.208845,-0.642858,...,-0.211763,-0.228663,-0.542464,-0.161780,-0.354038,-0.317835,-0.326248,-0.070785,-0.052444,5.510000
50%,84650.000000,0.025907,0.063052,0.177015,-0.022399,-0.054543,-0.273589,0.038361,0.021971,-0.049979,...,-0.062789,-0.030281,0.006057,-0.010666,0.041286,0.015550,-0.052173,0.001340,0.011317,22.000000
75%,139333.000000,1.317510,0.798376,1.024675,0.739895,0.611792,0.398343,0.567914,0.325913,0.599765,...,0.131815,0.185224,0.526075,0.148339,0.440253,0.350490,0.240273,0.090650,0.078043,77.070000
max,172792.000000,2.454930,22.057729,9.382558,16.875344,34.801666,73.301626,120.589494,20.007208,15.594995,...,38.117209,27.202839,8.361985,22.083545,4.584549,7.519589,3.517346,31.612198,33.847808,25691.160000


In [31]:
scalar = StandardScaler()   # Standardising Time and Amount in traning dataframe
X_val[['Time', 'Amount']] = scalar.fit_transform(X_val[['Time', 'Amount']])

scalar = StandardScaler()   #  Standardising Time and Amount in validation dataframe
X_val[['Time', 'Amount']] = scalar.fit_transform(X_val[['Time', 'Amount']])

In [32]:
df_train = pd.DataFrame(X_train, copy = True)  # creating a training dataframe for with all the features and label
df_train["Class"]=Y_train